# Инициализируем подключение к GigaChat

In [3]:
from dotenv import find_dotenv, load_dotenv
from langchain_gigachat import GigaChat

load_dotenv(find_dotenv())
giga = GigaChat(model="GigaChat-Max", profanity_check=False, verify_ssl_certs=False)
giga.invoke("Привет!").content

'Здравствуйте! Рад вас видеть. Как ваши дела?'

# Загружаем структурированные данные

In [5]:
from stuff_utils import StuffDesc
import json

with open('stuff_desc_list.json', 'r') as file:
    stuff_desc_list = json.load(file)

stuff_desc_objects = [StuffDesc(**desc) for desc in stuff_desc_list]
len(stuff_desc_objects)

48

# Создаем интсрумент поиска по базе данных

In [16]:
from typing import Literal, List
from langchain_core.tools import tool


@tool
def search_stuff(
    brand: str = None,
    max_price: float = None,
    min_store_temperature: int = None,
    max_store_temperature: int = None,
    shelf_life: int = None,
    type_of_packing: str = None,
    mass_fraction_of_fat_min: float = None,
    mass_fraction_of_fat_max: float = None
) -> List[StuffDesc]:
    """Выполняет поиск по базе данных продуктов.
type_of_packing - должно иметь только одно из этих значений: ["пластиковая бутылка", "стеклянная бутылка", "пакет", "железная банка", "алюминиевая банка", "коробка", "другое"]
"""
    results = []
    for stuff in stuff_desc_objects:
        if (brand is None or stuff.brand == brand) and \
           (max_price is None or stuff.price <= max_price) and \
           (min_store_temperature is None or stuff.min_store_temperature >= min_store_temperature) and \
           (max_store_temperature is None or stuff.max_store_temperature <= max_store_temperature) and \
           (shelf_life is None or stuff.shelf_life >= shelf_life) and \
           (type_of_packing is None or stuff.type_of_packing == type_of_packing) and \
           (mass_fraction_of_fat_min is None or stuff.mass_fraction_of_fat_min >= mass_fraction_of_fat_min) and \
           (mass_fraction_of_fat_max is None or stuff.mass_fraction_of_fat_max <= mass_fraction_of_fat_max):
            results.append(stuff)
    return str(results[0:10]) # Size limit

# Создаем ReAct агента с возможностью обращения к поиску

In [17]:
from langgraph.prebuilt import create_react_agent

tools = [search_stuff]
graph = create_react_agent(giga, tools=tools)

SYSTEM = """Ты помошник по поиску продуктов под запрос пользователя. 
Для поиска используй функцию для обращения к базе данных search_stuff.
Ты можешь вызвать функцию несколько раз, чтобы получить нужный результат.
Если результатов поиска больше 4, предупреди пользователя, что запрос слишком общий.
"""

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


def answer(question):
    inputs = {"messages": [(("system", SYSTEM)), ("user", question)]}
    print_stream(graph.stream(inputs, stream_mode="values"))

# Тестируем агента на референтных вопросах

In [18]:
answer("найди мне молоко с жирностью менее 3.9% в пакете дешевле 100 рублей")

================================ Human Message =================================

найди мне молоко с жирностью менее 3.9% в пакете дешевле 100 рублей
================================== Ai Message ==================================
Tool Calls:
  search_stuff (7524ee26-5bcb-4b5a-945d-becd503ec69d)
 Call ID: 7524ee26-5bcb-4b5a-945d-becd503ec69d
  Args:
    mass_fraction_of_fat_max: 3.9
    max_price: 100
    type_of_packing: пакет
================================= Tool Message =================================
Name: search_stuff

[StuffDesc(stuff_id='2', desc='Ультрапастеризованное молоко — молоко длительного хранения.', brand='Лазовское', price=99.1, min_store_temperature=2, max_store_temperature=25, shelf_life=180, type_of_packing='пакет', mass_fraction_of_fat_min=-1.0, mass_fraction_of_fat_max=-1.0), StuffDesc(stuff_id='3', desc='Ультрапастеризованное молоко — молоко длительного хранения.', brand='Лазовское', price=92.35, min_store_temperature=2, max_store_temperature=25, shelf_life=18

In [19]:
answer("Хочу заказать молоко не дороже 100руб, в бутылке. Какие есть варианты?")

================================ Human Message =================================

Хочу заказать молоко не дороже 100руб, в бутылке. Какие есть варианты?
================================== Ai Message ==================================
Tool Calls:
  search_stuff (f3be2368-e172-480a-b52f-547e8eb672d7)
 Call ID: f3be2368-e172-480a-b52f-547e8eb672d7
  Args:
    max_price: 100
    type_of_packing: пластиковая бутылка
================================= Tool Message =================================
Name: search_stuff

[StuffDesc(stuff_id='18', desc='Натуральное молоко. изготовлено по ГОСТ. Без добавлений.', brand='Домашкино', price=85.8, min_store_temperature=0, max_store_temperature=6, shelf_life=20, type_of_packing='пластиковая бутылка', mass_fraction_of_fat_min=3.2, mass_fraction_of_fat_max=3.2), StuffDesc(stuff_id='21', desc='О пользе коровьего молока еще с самого детства...', brand='Пестравка', price=91.3, min_store_temperature=0, max_store_temperature=6, shelf_life=16, type_of_packing='п

In [20]:
answer("Покажи мне молоко, которое можно хранить без холодильника, жирностью менее 2% и дешевле 90 руб.")

================================ Human Message =================================

Покажи мне молоко, которое можно хранить без холодильника, жирностью менее 2% и дешевле 90 руб.
================================== Ai Message ==================================
Tool Calls:
  search_stuff (ebe88542-b694-4299-90a2-bfc53858ae3f)
 Call ID: ebe88542-b694-4299-90a2-bfc53858ae3f
  Args:
    mass_fraction_of_fat_max: 2
    max_price: 90
    min_store_temperature: 0
================================= Tool Message =================================
Name: search_stuff

[StuffDesc(stuff_id='6', desc='Ультрапастеризованное молоко — молоко длительного хранения.', brand='Лазовское', price=74.7, min_store_temperature=2, max_store_temperature=25, shelf_life=180, type_of_packing='пакет', mass_fraction_of_fat_min=-1.0, mass_fraction_of_fat_max=-1.0)]
================================== Ai Message ==================================

Вот подходящее молоко: Ультрапастеризованное молоко от бренда Лазовское за 74.7

In [21]:
answer("Нужно молоко, которое можно храниться более 2 месяцев.")

================================ Human Message =================================

Нужно молоко, которое можно храниться более 2 месяцев.
================================== Ai Message ==================================
Tool Calls:
  search_stuff (74f115ac-453d-4134-b6dd-cb16a12d74dc)
 Call ID: 74f115ac-453d-4134-b6dd-cb16a12d74dc
  Args:
    shelf_life: 60
================================= Tool Message =================================
Name: search_stuff

[StuffDesc(stuff_id='2', desc='Ультрапастеризованное молоко — молоко длительного хранения.', brand='Лазовское', price=99.1, min_store_temperature=2, max_store_temperature=25, shelf_life=180, type_of_packing='пакет', mass_fraction_of_fat_min=-1.0, mass_fraction_of_fat_max=-1.0), StuffDesc(stuff_id='3', desc='Ультрапастеризованное молоко — молоко длительного хранения.', brand='Лазовское', price=92.35, min_store_temperature=2, max_store_temperature=25, shelf_life=180, type_of_packing='пакет', mass_fraction_of_fat_min=-1.0, mass_fraction_o